pmgus_dev_stuff.ipynb

In [20]:
import pandas as pd
import numpy as np
import requests
import os
import json

In [21]:
import dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
# TradingView custom columns setup
cols_list = [
    'Symbol', # ticker
    'Description', # something
    'marketCapType', 
    'Pre-market Change %', 
    'Pre-market Gap %', 
    'marketCapType',
    'Market capitalization',
    'Price', 
    'Pre-market Open', 
    'Industry', 
    'Index', 
    'Sector', 
    'Exchange',
    'Recent earnings date', 
    'Upcoming earnings date', 
    'Float shares outstanding', 
    'Average Volume 10 days',
    'Average Volume 30 days', 
    'Average Volume 90 days',
    'Relative Volume 1 day', 
    'Relative Volume 5 minutes', 
    'Relative Volume 30 minutes', 
    'Relative Volume at Time', 
    'Analyst Rating',
    'Technical Rating 5 minutes'
]

get pre-market price data for SOFI
-> do i need a websocket for pre-market data? after hours, post market ours data?
get pre-market clone data of TradingView
add features new and other ideas that we find

In [23]:
def get_snapshot(ticker, api_key):
    """
    Retrieves the snapshot data for the specified ticker from Polygon.io.
    """
    url = f"https://api.polygon.io/v2/snapshot/locale/us/markets/stocks/tickers/{ticker}?apiKey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        return data
    else:
        # Handle any errors
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [24]:
ticker = "SOFI"
api_key = os.getenv("polygon_api_key")  
snapshot_data = get_snapshot(ticker, api_key)

# Print the raw JSON response in a nice format
if snapshot_data:
    print(json.dumps(snapshot_data, indent=2))
    
    # If you want to extract specific fields, for example the last trade price:
    if 'ticker' in snapshot_data and 'lastTrade' in snapshot_data['ticker']:
        last_trade_price = snapshot_data['ticker']['lastTrade'].get('p')
        print(f"\nThe most recent trade price for {ticker} is: {last_trade_price}")
    else:
        print("\nNo 'lastTrade' data found in the snapshot response.")

{
  "ticker": {
    "ticker": "SOFI",
    "todaysChangePerc": 3.807740324594254,
    "todaysChange": 0.6099999999999994,
    "updated": 1735261200000000000,
    "day": {
      "o": 15.98,
      "h": 16.68,
      "l": 15.88,
      "c": 16.6,
      "v": 36892700.0,
      "vw": 16.4665
    },
    "lastQuote": {
      "P": 16.63,
      "S": 49,
      "p": 16.61,
      "s": 1,
      "t": 1735261199752001700
    },
    "lastTrade": {
      "c": [
        12
      ],
      "i": "8163",
      "p": 16.63,
      "s": 308,
      "t": 1735261194971047072,
      "x": 11
    },
    "min": {
      "av": 36892700.0,
      "t": 1735261140000,
      "n": 16,
      "o": 16.63,
      "h": 16.63,
      "l": 16.59,
      "c": 16.63,
      "v": 1188,
      "vw": 16.6162
    },
    "prevDay": {
      "o": 15.73,
      "h": 16.03,
      "l": 15.57,
      "c": 16.02,
      "v": 15729659.0,
      "vw": 15.8539
    }
  },
  "status": "OK",
  "request_id": "500a970f31cff9f15548b797d583ca34"
}

The most recent trad

DEV 2

In [25]:
def get_enhanced_snapshot(ticker, api_key):
    """
    Enhanced snapshot function that includes pre-market data and error handling
    """
    base_url = "https://api.polygon.io/v2"
    
    # Get snapshot data
    snapshot_url = f"{base_url}/snapshot/locale/us/markets/stocks/tickers/{ticker}"
    
    # Get previous day's close and pre-market data
    aggs_url = f"{base_url}/aggs/ticker/{ticker}/range/1/minute/previous"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    try:
        snapshot_response = requests.get(snapshot_url, headers=headers)
        snapshot_response.raise_for_status()
        snapshot_data = snapshot_response.json()
        
        # Calculate pre-market metrics
        current_price = snapshot_data['ticker']['lastTrade']['p']
        prev_close = snapshot_data['ticker']['prevDay']['c']
        
        # Calculate pre-market changes
        pm_change = current_price - prev_close
        pm_change_percent = (pm_change / prev_close) * 100
        
        # Enhance the snapshot data with pre-market metrics
        enhanced_data = {
            'symbol': ticker,
            'current_price': current_price,
            'prev_close': prev_close,
            'pm_change': pm_change,
            'pm_change_percent': pm_change_percent,
            'volume': snapshot_data['ticker']['day']['v'],
            'vwap': snapshot_data['ticker']['day']['vw'],
            'timestamp': snapshot_data['ticker']['updated']
        }
        
        return enhanced_data
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {ticker}: {str(e)}")
        return None

In [26]:
def build_screening_df(tickers, api_key):
    """
    Build a DataFrame with screening data for multiple tickers
    """
    results = []
    
    for ticker in tickers:
        data = get_enhanced_snapshot(ticker, api_key)
        if data:
            results.append(data)
    
    df = pd.DataFrame(results)
    
    # Add additional calculated columns
    if not df.empty:
        df['rel_volume'] = df['volume'] / df['volume'].mean()
        # YOU CAN ADD STUFF HERE LATER        
    return df

# Usage example:
tickers = ['SOFI', 'AAPL', 'MSFT']  # Add more tickers as needed
screening_df = build_screening_df(tickers, api_key)

In [27]:
display(screening_df)

,symbol,current_price,prev_close,pm_change,pm_change_percent,volume,vwap,timestamp,rel_volume,price_range
0,SOFI,16.6300,16.02,0.6100,3.807740,36892700.0,16.4665,1735261200000000000,1.556929,NaN
1,AAPL,259.0600,258.20,0.8600,0.333075,26225997.0,258.9280,1735261140000000000,1.106777,NaN
2,MSFT,437.4596,439.33,-1.8704,-0.425739,7968758.0,438.6447,1735261200000000000,0.336294,NaN
